# Hyperparameter Tuning
Find the most acceptable parameter

In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy

## Data for Tuning

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

In [3]:
# setting
hotel,tur,timematrix = generate_data(n=30,random_state=30)
travel_days = 3
dwaktu,dtarif,drating = 1,1,1

C:\Users\rahma\Documents\kuliah\thesis\rahmat_code\travel-itinerary-recommendation\sql_connection.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataset = pd.read_sql(query, mydb)


## Maximum Iteration

In [5]:
# List of iterations
iterations = [100, 200, 300]

#list of algorithms
for n in iterations:
    print(n)
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = n,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")
    
    print()
    print("ACS BSO - TSP")
    start = time.time()
    acsbso_tsp = ACSBSO_TSP(max_iter_acs = n,random_state=100)
    acsbso_tsp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_tsp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")


100
ACS BSO - VRP
fitness = 0.6880194416426301
time = 17.9234721660614

ACS BSO - TSP
fitness = 0.616598958067212
time = 27.046855688095093
200
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.85939645767212

ACS BSO - TSP
fitness = 0.616598958067212
time = 31.444244146347046
300
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.638370990753174

ACS BSO - TSP
fitness = 0.5999078155824186
time = 35.6239914894104


## Tuning Hybrid ACS BSO (VRP/TSP)

### Find Initial Pheromone

In [12]:
#find init pheromone
init_pheromone = [0.1,0.5,1]

for pheromone in init_pheromone:
    print(f"pheromone = {pheromone}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=pheromone,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

pheromone = 0.1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.228255033493042
pheromone = 0.5
ACS BSO - VRP
fitness = 0.6928586771840739
time = 29.90246319770813
pheromone = 1
ACS BSO - VRP
fitness = 0.6791632749966082
time = 8.105718612670898


### Find Alpha_t

In [13]:
#find alpha_t
alpha_t_ls = [1,3,5]

for alpha_t in alpha_t_ls:
    print(f"alpha_t = {alpha_t}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=alpha_t,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

alpha_t = 1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.442343711853027
alpha_t = 3
ACS BSO - VRP
fitness = 0.6766389401310037
time = 8.094337224960327
alpha_t = 5
ACS BSO - VRP
fitness = 0.665530706800548
time = 5.593754768371582


### Find beta

In [10]:
#find beta
beta_ls = [1,3,5]

for beta in beta_ls:
    print(f"beta = {beta}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=beta,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

beta = 1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.167970895767212
beta = 3
ACS BSO - VRP
fitness = 0.6930605939060044
time = 25.666985750198364
beta = 5
ACS BSO - VRP
fitness = 0.690109630292515
time = 17.28569984436035


### Find q0

In [14]:
#find beta
q0_ls = [0.1,0.5,0.9]

for q0 in q0_ls:
    print(f"q0 = {q0}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,q0=q0,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

q0 = 0.1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 26.1429705619812
q0 = 0.5
ACS BSO - VRP
fitness = 0.6926490712722596
time = 17.463536500930786
q0 = 0.9
ACS BSO - VRP
fitness = 0.6900256144821362
time = 13.919016361236572


### Find Rho

In [15]:
#find rho
rho_ls = [0.1,0.5,0.9]

for rho in rho_ls:
    print(f"rho = {rho}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,q0=0.1,rho=rho,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

rho = 0.1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.320417165756226
rho = 0.5
ACS BSO - VRP
fitness = 0.6882252029595025
time = 16.452585220336914
rho = 0.9
ACS BSO - VRP
fitness = 0.6916717050171156
time = 14.610160112380981


### Find Alpha

In [16]:
#find alpha
alpha_ls = [0.1,0.5,0.9]

for alpha in alpha_ls:
    print(f"alpha = {alpha}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,q0=0.1,rho=0.1,alpha=alpha,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

alpha = 0.1
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.227779626846313
alpha = 0.5
ACS BSO - VRP
fitness = 0.6913956385229677
time = 21.391562461853027
alpha = 0.9
ACS BSO - VRP
fitness = 0.6885595650994202
time = 10.09394884109497


### Find Num Ant

In [18]:
#find num_ant
num_ant_ls = [30,60,90]

for num_ant in num_ant_ls:
    print(f"num_ant = {num_ant}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,q0=0.1,rho=0.1,alpha=0.1,num_ant=num_ant,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

num_ant = 30
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.02150821685791
num_ant = 60
ACS BSO - VRP
fitness = 0.6921139563220308
time = 18.742931365966797
num_ant = 90
ACS BSO - VRP
fitness = 0.6868918118918118
time = 16.177350759506226


### Find P0

In [24]:
#find p0
p0_ls = [0.1,0.5,0.9]

for p0 in p0_ls:
    print(f"p0 = {p0}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=p0,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p0 = 0.1
ACS BSO - VRP
fitness = 0.6922671133980658
time = 20.0521981716156
p0 = 0.5
ACS BSO - VRP
fitness = 0.6924740386265574
time = 24.970614910125732
p0 = 0.9
ACS BSO - VRP
fitness = 0.6921861083092966
time = 11.906808376312256


### Find P1

In [27]:
#find p1
p1_ls = [0.4,0.5,0.6]

for p1 in p1_ls:
    print(f"p1 = {p1}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=0.5,p1=p1,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p1 = 0.4
ACS BSO - VRP
fitness = 0.6924740386265574
time = 24.96790623664856
p1 = 0.5
ACS BSO - VRP
fitness = 0.6916509933590679
time = 18.49229884147644
p1 = 0.6
ACS BSO - VRP
fitness = 0.6910544210664984
time = 12.943957567214966


### Find P2

In [28]:
#find p2
p2_ls = [0.4,0.5,0.6]

for p2 in p2_ls:
    print(f"p2 = {p2}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=0.5,p1=0.4,p2=p2,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p2 = 0.4
ACS BSO - VRP
fitness = 0.6924740386265574
time = 24.43697476387024
p2 = 0.5
ACS BSO - VRP
fitness = 0.6934514048817015
time = 32.5661199092865
p2 = 0.6
ACS BSO - VRP
fitness = 0.6889265027204985
time = 11.985989809036255


### Find P3

In [29]:
#find p3
p3_ls = [0.4,0.5,0.6]

for p3 in p3_ls:
    print(f"p3 = {p3}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=0.5,p1=0.4,p2=0.4,
                            p3=p3,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p3 = 0.4
ACS BSO - VRP
fitness = 0.6909772605726711
time = 13.551257848739624
p3 = 0.5
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.217138290405273
p3 = 0.6
ACS BSO - VRP
fitness = 0.6899741741529182
time = 23.21198010444641


### Find P4

In [31]:
#find p4
p4_ls = [0.4,0.5,0.6]

for p4 in p4_ls:
    print(f"p4 = {p4}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=0.5,p1=0.4,p2=0.4,
                            p3=0.5,p4=p4,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

p4 = 0.4
ACS BSO - VRP
fitness = 0.6916717050171156
time = 13.118225812911987
p4 = 0.5
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.095247268676758
p4 = 0.6
ACS BSO - VRP
fitness = 0.6901022588522586
time = 17.968683004379272


### Find Max Iter BSO

In [33]:
#find p4
max_iter_bso_ls = [15,20,30]

for max_iter_bso in max_iter_bso_ls:
    print(f"max_iter_bso = {max_iter_bso}")
    print("ACS BSO - VRP")
    start = time.time()
    acsbso_vrp = ACSBSO_VRP(max_iter_acs = 200,init_pheromone=0.1,alpha_t=1,beta=1,
                            q0=0.1,rho=0.1,alpha=0.1,num_ant=30,p0=0.5,p1=0.4,p2=0.4,
                            p3=0.5,p4=0.5,max_iter_bso=max_iter_bso,random_state=100)
    acsbso_vrp.set_model(tur,hotel,timematrix,travel_days=travel_days,degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    fitness = acsbso_vrp.construct_solution()[-1]
    end = time.time()
    print(f"fitness = {fitness}")
    print(f"time = {end-start}")
    print("==============================================")

max_iter_bso = 15
ACS BSO - VRP
fitness = 0.6924740386265574
time = 25.019355535507202
max_iter_bso = 20
ACS BSO - VRP
fitness = 0.6905728460507617
time = 16.510849952697754
max_iter_bso = 30
ACS BSO - VRP
fitness = 0.6903907052932242
time = 19.568053245544434
